In [ ]:
!pip install torchaudio soundfile librosa
!pip install datasets -U

In [ ]:
!pip uninstall -y datasets
!pip install datasets==2.21.0
!pip install transformers accelerate soundfile librosa jiwer torchaudio
!apt install git-lfs

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [ ]:
#wav2vec2
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import load_dataset, Audio
import re
import json
import shutil
from google.colab import drive
from transformers import (
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer
)


drive.mount('/content/drive')

MODEL_ID = "facebook/wav2vec2-large-xlsr-53"
print("Loading Russian dataset (Golos - 1000 samples)...")
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split="train[:1000]")


chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\]]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

dataset = dataset.map(remove_special_characters)

def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_dict = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names)
vocab_list = list(set(vocab_dict["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

encoded_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names, num_proc=1)

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, padding=self.padding, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, padding=self.padding, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


model = Wav2Vec2ForCTC.from_pretrained(
    MODEL_ID,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_encoder()

training_args = TrainingArguments(
    output_dir="./wav2vec2-russian-master",
    group_by_length=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    eval_strategy="steps",
    num_train_epochs=15,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=300,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
    tokenizer=processor.feature_extractor,
)

print("Starting Master Training...")
trainer.train()


print("Saving model to Google Drive...")
drive_path = "/content/drive/My Drive/wav2vec2-russian-master"
trainer.save_model(drive_path)
processor.save_pretrained(drive_path)
print(f"DONE! Model safe at: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading Russian dataset (Golos - 1000 samples)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/backends/cudnn/__init__.py:145: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  torch._C._get_cudnn_allow_tf32(),


Step,Training Loss,Validation Loss
500,3.168800,3.103037
1000,1.213500,0.664822
1500,0.755100,0.334109


Saving model to Google Drive...
DONE! Model safe at: /content/drive/My Drive/wav2vec2-russian-master


In [ ]:
# continue wav2vec2

args_refine = TrainingArguments(
    output_dir="./wav2vec2-russian-master-refined",
    group_by_length=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    eval_strategy="steps",
    num_train_epochs=20,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    warmup_steps=100,
    save_total_limit=1,
)

trainer_refine = Trainer(
    model=model,
    data_collator=data_collator,
    args=args_refine,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
    tokenizer=processor.feature_extractor,
)

print("Starting Refinement Training (20 more epochs)...")
trainer_refine.train()

print("Saving refined model to Google Drive...")
drive_path_refined = "/content/drive/My Drive/wav2vec2-russian-refined"
trainer_refine.save_model(drive_path_refined)
processor.save_pretrained(drive_path_refined)
print(f"DONE! Refined model safe at: {drive_path_refined}")

/tmp/ipython-input-3901452281.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_refine = Trainer(


Starting Refinement Training (20 more epochs)...


Step,Training Loss,Validation Loss
500,0.526900,0.174310
1000,0.365800,0.116004
1500,0.303400,0.077120
2000,0.347400,0.060028
2500,0.344700,0.053350


Saving refined model to Google Drive...
DONE! Refined model safe at: /content/drive/My Drive/wav2vec2-russian-refined


In [ ]:
import torch
import librosa
from google.colab import files


print("Please upload your audio file now:")
uploaded = files.upload()
filename = next(iter(uploaded))

print(f"Processing {filename}...")
speech, rate = librosa.load(filename, sr=16000)

input_values = processor(speech, sampling_rate=16000, return_tensors="pt").input_values.to("cuda")

print("Transcribing...")
with torch.no_grad():
    logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(pred_ids)[0]

print("\n" + "="*40)
print("TARGET: Съешь же ещё этих мягких французских булок, да выпей чаю.")
print(f"WAV2VEC2: {transcription}")
print("="*40)

Please upload your audio file now:


Saving pangramm6.mp4 to pangramm6 (3).mp4
Processing pangramm6 (3).mp4...


/tmp/ipython-input-2485829679.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, rate = librosa.load(filename, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribing...

TARGET: Съешь же ещё этих мягких французских булок, да выпей чаю.
WAV2VEC2: сешь жиеще этих мяких француских булак довы пойчаю


In [3]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset, Audio
from jiwer import wer, cer
from tqdm.auto import tqdm
import re
import io
import soundfile as sf
import torchaudio

MODEL_NAME = "openai/whisper-small"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🚀 Loading Whisper ({MODEL_NAME}) on {DEVICE}...")

processor = WhisperProcessor.from_pretrained(MODEL_NAME)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

print("Loading Golos Dataset (First 100 samples)...")
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split="test[:100]")

dataset = dataset.cast_column("audio", Audio(decode=False))

chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\]]'
def normalize_text(text):
    if not text: return ""
    text = re.sub(chars_to_ignore_regex, "", text.lower())
    return " ".join(text.split())

predictions = []
references = []

print(f"\nRunning Inference...")

resampler = None

for item in tqdm(dataset):
    audio_bytes = item["audio"]["bytes"]
    audio_array, sample_rate = sf.read(io.BytesIO(audio_bytes))

    audio_tensor = torch.tensor(audio_array, dtype=torch.float32)

    if sample_rate != 16000:
        if resampler is None or resampler.orig_freq != sample_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        audio_tensor = resampler(audio_tensor)

    input_features = processor(audio_tensor, sampling_rate=16000, return_tensors="pt").input_features.to(DEVICE)

    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    ref_norm = normalize_text(item["transcription"])
    pred_norm = normalize_text(transcription)

    references.append(ref_norm)
    predictions.append(pred_norm)

final_wer = wer(references, predictions)
final_cer = cer(references, predictions)

print("\n" + "="*50)
print(f"WHISPER ({MODEL_NAME}) RESULTS")
print("="*50)
print(f"WER: {final_wer:.2%}")
print(f"CER: {final_cer:.2%}")
print("="*50)

for i in range(5):
    print(f"Ref:  {references[i]}")
    print(f"Pred: {predictions[i]}")
    print("-" * 30)

🚀 Loading Whisper (openai/whisper-small) on cuda...
Loading Golos Dataset (First 100 samples)...

Running Inference...


  0%|          | 0/100 [00:00<?, ?it/s]

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



WHISPER (openai/whisper-small) RESULTS
WER: 34.86%
CER: 18.86%
Ref:  шестьдесят тысяч тенге сколько будет стоить
Pred: 60 тысяч тенге сколько будет стоить
------------------------------
Ref:  покажи мне на смотрешке телеканал синергия тв
Pred: покажи мне на смотрёжке телеканал синергии тв
------------------------------
Ref:  заказать яблоки зеленые
Pred: закосот с яблоки зеленую
------------------------------
Ref:  алиса закажи килограммовый торт графские развалины
Pred: алиса закажи килограммовый торт графские развалены
------------------------------
Ref:  ищи телеканал про бизнес на тиви
Pred: ищи телеканал про бизнес на тв
------------------------------


In [4]:
import torch
import torchaudio
import librosa
import soundfile as sf
import io
import re
from datasets import load_dataset, Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from jiwer import wer, cer
from tqdm.auto import tqdm

MODEL_ID = "bond005/wav2vec2-large-ru-golos"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_SAMPLES = 100

print(f"🚀 Loading Specialist Model ({MODEL_ID}) on {DEVICE}...")

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(DEVICE)

print("Loading Golos Dataset...")
dataset = load_dataset("bond005/sberdevices_golos_10h_crowd", split=f"test[:{NUM_SAMPLES}]")
dataset = dataset.cast_column("audio", Audio(decode=False))

chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\]]'
def normalize_text(text):
    if not text: return ""
    text = re.sub(chars_to_ignore_regex, "", text.lower())
    return " ".join(text.split())

predictions = []
references = []

print(f"\nRunning Inference...")
resampler = torchaudio.transforms.Resample(48000, 16000).to(DEVICE)

for item in tqdm(dataset):
    audio_bytes = item["audio"]["bytes"]
    audio_array, sample_rate = sf.read(io.BytesIO(audio_bytes))
    audio_tensor = torch.tensor(audio_array, dtype=torch.float32).to(DEVICE)

    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000).to(DEVICE)
        audio_tensor = resampler(audio_tensor)

    if audio_tensor.ndim == 1:
        audio_tensor = audio_tensor.unsqueeze(0)

    inputs = processor(audio_tensor.squeeze().cpu().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    pred_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(pred_ids)[0]

    ref_norm = normalize_text(item["transcription"])
    pred_norm = normalize_text(transcription)

    references.append(ref_norm)
    predictions.append(pred_norm)

final_wer = wer(references, predictions)
final_cer = cer(references, predictions)

print("\n" + "="*50)
print(f"SPECIALIST ({MODEL_ID}) RESULTS")
print("="*50)
print(f"WER: {final_wer:.2%}")
print(f"CER: {final_cer:.2%}")
print("="*50)

for i in range(5):
    print(f"Ref:  {references[i]}")
    print(f"Pred: {predictions[i]}")
    print("-" * 30)

🚀 Loading Specialist Model (bond005/wav2vec2-large-ru-golos) on cuda...


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Loading Golos Dataset...

Running Inference...


  0%|          | 0/100 [00:00<?, ?it/s]


SPECIALIST (bond005/wav2vec2-large-ru-golos) RESULTS
WER: 8.17%
CER: 1.51%
Ref:  шестьдесят тысяч тенге сколько будет стоить
Pred: шестьдесят тысяч тенге сколько будет стоить
------------------------------
Ref:  покажи мне на смотрешке телеканал синергия тв
Pred: покажи мне на смотрешке телеканал синергия тв
------------------------------
Ref:  заказать яблоки зеленые
Pred: заказать с яблоки зеленые
------------------------------
Ref:  алиса закажи килограммовый торт графские развалины
Pred: алиса закажи килограммовый торт графские развалинные
------------------------------
Ref:  ищи телеканал про бизнес на тиви
Pred: ищи телеканал про бизнес на тиви
------------------------------


In [5]:
import torch
import torchaudio
import librosa
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from google.colab import files

MODEL_ID = "bond005/wav2vec2-large-ru-golos"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PANGRAM_TEXT = "съешь же ещё этих мягких французских булок да выпей чаю"

print(f"🚀 Loading Champion Model ({MODEL_ID}) on {DEVICE}...")

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(DEVICE)

print("\n Upload your Pangram Audio File (wav/mp3)...")
uploaded = files.upload()
filename = next(iter(uploaded))

print(f"Processing {filename}...")
speech, sr = librosa.load(filename, sr=16000)

input_values = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest").input_values.to(DEVICE)

model.eval()
with torch.no_grad():
    logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(pred_ids)[0]

print("\n" + "="*50)
print("PANGRAM TEST RESULTS")
print("="*50)
print(f"TARGET:     {PANGRAM_TEXT}")
print(f"PREDICTION: {transcription}")
print("="*50)

🚀 Loading Champion Model (bond005/wav2vec2-large-ru-golos) on cuda...

 Upload your Pangram Audio File (wav/mp3)...


Saving pangramm6.mp4 to pangramm6.mp4
Processing pangramm6.mp4...


/tmp/ipython-input-1390134246.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(filename, sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



PANGRAM TEST RESULTS
TARGET:     съешь же ещё этих мягких французских булок да выпей чаю
PREDICTION: сешже еще этих мягких французских булок да вы пойчаю
